## The CBS Secret Sauce Investigation: NCIS:NLP -- EDA

#### Creator: Mitch Brinkman

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import patsy
import re
import pickle
import string
from sklearn.feature_extraction.text import CountVectorizer
from collections import Counter
from sklearn.feature_extraction import text
from wordcloud import WordCloud, STOPWORDS
from gensim import corpora, models, similarities, matutils
from sklearn import datasets
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

### Word Frequecy Analysis

In [4]:
shield_data = pd.read_pickle('./data/dtm/shield_dtm.pkl')
shield_data = shield_data.transpose()
shield_data.tail(30)

ep_id,the_shield_season-1/episode-1-Pilot,the_shield_season-1/episode-2-Our_Gang,the_shield_season-1/episode-3-The_Spread,the_shield_season-1/episode-4-Dawg_Days,the_shield_season-1/episode-5-Blowback,the_shield_season-1/episode-6-Cherrypoppers,the_shield_season-1/episode-7-Pay_in_Pain,the_shield_season-1/episode-8-Cupid__Psycho,the_shield_season-1/episode-9-Throwaway,the_shield_season-1/episode-10-Dragonchasers,...,the_shield_season-7/episode-4-Genocide,the_shield_season-7/episode-5-Game_Face,the_shield_season-7/episode-6-Animal_Control,the_shield_season-7/episode-7-Bitches_Brew,the_shield_season-7/episode-8-Parricide,the_shield_season-7/episode-9-Moving_Day,the_shield_season-7/episode-10-Party_Line,the_shield_season-7/episode-11-Petty_Cash,the_shield_season-7/episode-12-Possible_Kill_Screen,the_shield_season-7/episode-13-Family_Meeting
zal,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zamora,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zamorski,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zapper,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zavala,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zayed,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zeal,0,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zeborio,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zellie,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zellies,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
ncis_data = pd.read_pickle('./data/dtm/ncis_dtm.pkl')
ncis_data = ncis_data.transpose()
ncis_data.tail(30)

ep_id,ncis_season-1/episode-1-Yankee_White,ncis_season-1/episode-2-Hung_Out_to_Dry,ncis_season-1/episode-3-Seadog,ncis_season-1/episode-4-The_Immortals,ncis_season-1/episode-5-The_Curse,ncis_season-1/episode-6-High_Seas,ncis_season-1/episode-7-Sub_Rosa,ncis_season-1/episode-8-Minimum_Security,ncis_season-1/episode-9-Marine_Down,ncis_season-1/episode-10-Left_for_Dead,...,ncis_season-17/episode-6-Institutionalized,ncis_season-17/episode-8-No_Vacancy,ncis_season-17/episode-9-IRL,ncis_season-17/episode-10-The_North_Pole,ncis_season-17/episode-11-In_the_Wind,ncis_season-17/episode-12-Flight_Plan,ncis_season-17/episode-13-Sound_Off,ncis_season-17/episode-14-On_Fire,ncis_season-17/episode-15-Lonely_Hearts,ncis_season-17/episode-16-Ephemera
zuse,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zuzim,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
zygomatic,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ándale,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
éses,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ﬁancé,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ﬁancés,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ﬁbre,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ﬁeld,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ﬁery,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [6]:
# Find the top 30 words said by each comedian
top_dict1 = {}
for c in shield_data.columns:
    top = shield_data[c].sort_values(ascending=False).head(30)
    top_dict1[c]= list(zip(top.index, top.values))

top_dict1

{'the_shield_season-1/episode-1-Pilot': [('got', 61),
  ('know', 42),
  ('dont', 35),
  ('yeah', 34),
  ('like', 33),
  ('im', 33),
  ('hey', 29),
  ('guy', 27),
  ('na', 27),
  ('right', 25),
  ('just', 23),
  ('wa', 23),
  ('jenny', 22),
  ('come', 21),
  ('think', 21),
  ('gon', 20),
  ('girl', 18),
  ('thing', 17),
  ('mean', 15),
  ('let', 15),
  ('man', 14),
  ('did', 14),
  ('oh', 14),
  ('make', 13),
  ('cop', 13),
  ('sure', 13),
  ('look', 13),
  ('youre', 12),
  ('people', 12),
  ('time', 12)],
 'the_shield_season-1/episode-2-Our_Gang': [('dont', 43),
  ('wa', 39),
  ('know', 37),
  ('terry', 33),
  ('right', 30),
  ('im', 29),
  ('didnt', 29),
  ('like', 29),
  ('come', 28),
  ('got', 28),
  ('guy', 27),
  ('vic', 25),
  ('na', 25),
  ('yeah', 24),
  ('thats', 20),
  ('gon', 19),
  ('man', 18),
  ('just', 18),
  ('look', 17),
  ('shot', 17),
  ('let', 16),
  ('did', 16),
  ('drink', 16),
  ('cop', 14),
  ('make', 14),
  ('good', 14),
  ('dead', 13),
  ('tell', 13),
  ('time

In [8]:
# Find the top 30 words said by each comedian
top_dict = {}
for c in ncis_data.columns:
    top = ncis_data[c].sort_values(ascending=False).head(30)
    top_dict[c]= list(zip(top.index, top.values))

top_dict

{'ncis_season-1/episode-1-Yankee_White': [('president', 52),
  ('wa', 43),
  ('agent', 33),
  ('got', 23),
  ('know', 23),
  ('body', 23),
  ('im', 22),
  ('commander', 20),
  ('yeah', 19),
  ('dont', 19),
  ('think', 19),
  ('gibbs', 18),
  ('did', 18),
  ('sir', 18),
  ('todd', 17),
  ('air', 17),
  ('right', 17),
  ('force', 16),
  ('like', 16),
  ('youre', 16),
  ('ducky', 14),
  ('okay', 14),
  ('na', 14),
  ('plane', 13),
  ('yes', 13),
  ('just', 12),
  ('oh', 12),
  ('trapp', 11),
  ('service', 11),
  ('secret', 11)],
 'ncis_season-1/episode-2-Hung_Out_to_Dry': [('wa', 61),
  ('sir', 48),
  ('chute', 36),
  ('know', 35),
  ('dont', 27),
  ('corporal', 27),
  ('yeah', 27),
  ('got', 26),
  ('like', 24),
  ('didnt', 22),
  ('did', 22),
  ('thumper', 21),
  ('marine', 21),
  ('ramsey', 20),
  ('na', 20),
  ('jump', 19),
  ('im', 19),
  ('time', 18),
  ('youre', 18),
  ('gon', 17),
  ('reserve', 15),
  ('key', 15),
  ('fuentes', 15),
  ('thats', 15),
  ('sergeant', 15),
  ('agent',

In [ ]:
for ep_id, top_words in top_dict1.items():
    print(ep_id)
    print(', '.join([word for word, count in top_words[0:5]]))
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

In [9]:
for ep_id, top_words in top_dict.items():
    print(ep_id)
    print(', '.join([word for word, count in top_words[0:5]]))
    print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

ncis_season-1/episode-1-Yankee_White
president, wa, agent, got, know
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ncis_season-1/episode-2-Hung_Out_to_Dry
wa, sir, chute, know, dont
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ncis_season-1/episode-3-Seadog
wa, boat, got, know, like
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ncis_season-1/episode-4-The_Immortals
wa, macdonald, sword, know, im
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ncis_season-1/episode-5-The_Curse
wa, lieutenant, know, pod, yeah
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ncis_season-1/episode-6-High_Seas
wa, sir, know, got, deck
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ncis_season-1/episode-7-Sub_Rosa
wa, sir, aye, agent, dont
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ncis_season-1/episode-8-Minimum_Security
wa, said, nasser, know, yeah
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ncis_season-1/episode-9-Marine_Down
wa, gibbs, know, dont, im
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ncis_season-1/episode-10-Left_for_Dead
know, wa, yeah, dont, did
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
ncis_season-1/episode-11-Eye_Spy
wa, 

In [ ]:
# Look at the most common top words
shield_words = []
for ep_id in ncis_data.columns:
    top = [word for (word, count) in top_dict[ep_id]]
    for t in top:
        words.append(t)
        
shield_words

In [10]:
# Look at the most common top words
ncis_words = []
for ep_id in ncis_data.columns:
    top = [word for (word, count) in top_dict[ep_id]]
    for t in top:
        words.append(t)
        
ncis_words

['president',
 'wa',
 'agent',
 'got',
 'know',
 'body',
 'im',
 'commander',
 'yeah',
 'dont',
 'think',
 'gibbs',
 'did',
 'sir',
 'todd',
 'air',
 'right',
 'force',
 'like',
 'youre',
 'ducky',
 'okay',
 'na',
 'plane',
 'yes',
 'just',
 'oh',
 'trapp',
 'service',
 'secret',
 'wa',
 'sir',
 'chute',
 'know',
 'dont',
 'corporal',
 'yeah',
 'got',
 'like',
 'didnt',
 'did',
 'thumper',
 'marine',
 'ramsey',
 'na',
 'jump',
 'im',
 'time',
 'youre',
 'gon',
 'reserve',
 'key',
 'fuentes',
 'thats',
 'sergeant',
 'agent',
 'yes',
 'just',
 'rigger',
 'come',
 'wa',
 'boat',
 'got',
 'know',
 'like',
 'drug',
 'dont',
 'commander',
 'right',
 'yeah',
 'just',
 'tony',
 'guy',
 'did',
 'gibbs',
 'didnt',
 'im',
 'agent',
 'want',
 'power',
 'van',
 'time',
 'number',
 'theyre',
 'look',
 'think',
 'come',
 'man',
 'good',
 'okay',
 'wa',
 'macdonald',
 'sword',
 'know',
 'im',
 'like',
 'sir',
 'seaman',
 'yeah',
 'ship',
 'just',
 'think',
 'youre',
 'officer',
 'na',
 'got',
 'did',


In [ ]:
Counter(shield_words).most_common()

In [11]:
# Let's aggregate this list and identify the most common words along with how many episodes they occur in
Counter(ncis_words).most_common()

[('wa', 377),
 ('know', 376),
 ('im', 375),
 ('dont', 373),
 ('got', 367),
 ('just', 364),
 ('like', 352),
 ('yeah', 346),
 ('right', 325),
 ('thats', 311),
 ('did', 297),
 ('youre', 290),
 ('gibbs', 279),
 ('think', 261),
 ('oh', 232),
 ('time', 200),
 ('na', 193),
 ('okay', 185),
 ('didnt', 182),
 ('want', 178),
 ('agent', 175),
 ('look', 164),
 ('good', 159),
 ('uh', 157),
 ('gon', 140),
 ('guy', 138),
 ('mcgee', 135),
 ('need', 124),
 ('tell', 114),
 ('bos', 100),
 ('let', 98),
 ('hey', 93),
 ('say', 89),
 ('come', 86),
 ('yes', 72),
 ('tony', 72),
 ('going', 69),
 ('mean', 64),
 ('ha', 63),
 ('make', 52),
 ('thing', 47),
 ('officer', 46),
 ('lieutenant', 44),
 ('year', 41),
 ('said', 41),
 ('way', 41),
 ('dinozzo', 38),
 ('ziva', 38),
 ('shes', 37),
 ('body', 33),
 ('man', 33),
 ('petty', 33),
 ('abby', 33),
 ('car', 33),
 ('mc', 33),
 ('really', 32),
 ('commander', 31),
 ('marine', 30),
 ('sir', 29),
 ('director', 29),
 ('people', 28),
 ('talk', 27),
 ('ill', 24),
 ('night', 23),